In [1]:
#import dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [28]:
 #Read the data as a CSV file
data1 = pd.read_csv(Path('ml_df.csv'))
data1['GRADUATED2'].value_counts()

Graduated                2534
Terminated/Discharged    1562
Name: GRADUATED2, dtype: int64

In [3]:
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

In [4]:
#create features

X = pd.get_dummies(data1, columns= ['PROGRAM_TYPE', 'RISK_LEVEL3']).drop('GRADUATED2',axis=1)

y = data1['GRADUATED2']

X = X.drop(['PROGRAM_TYPE_Family Treatment', 'PROGRAM_TYPE_Felony Drug','PROGRAM_TYPE_Juvenile Drug',
            'PROGRAM_TYPE_Mental Health', 'PROGRAM_TYPE_Veterans Court', 'RISK_LEVEL3_low',
            'RISK_LEVEL3_moderate'           
                      ], axis=1)
X.head()

,No_SANC_INCENT,No_TREATMENTS,No_DRUG_TEST,No_HEARING,AGE,DAYS_TO_ACCEPTANCE,PROGRAM_TYPE_DUI Courts,RISK_LEVEL3_high
0,1,1,1,0,35.468847,5.0,0,1
1,0,0,21,1,35.468847,5.0,0,1
2,1,1,5,3,32.917170,75.0,0,0
3,1,1,0,0,32.509230,157.0,0,1
4,0,0,6,1,32.509230,157.0,0,1


In [5]:
# check version number
# Oversample and plot imbalanced dataset with SMOTE
from collections import Counter
import imblearn
print(imblearn.__version__)



0.8.1


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23, stratify=y)

In [7]:
from imblearn.over_sampling import SMOTE
# summarize class distribution
counter = Counter(y_train)
print('Before', counter)
oversample = SMOTE()
X_train_sm, y_train_sm = oversample.fit_resample(X_train, y_train)
counter = Counter(y_train_sm)
print('After', counter)

Before Counter({'Graduated': 1901, 'Terminated/Discharged': 1171})
After Counter({'Graduated': 1901, 'Terminated/Discharged': 1901})


In [14]:
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier
import pickle
#fit the model to the training set
gbm_model = GradientBoostingClassifier(random_state=42)
#Train our model using the training data: output training and testing scores



from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 40)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [3, 4, 5, 6, 7, 8]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 4, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 5]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}



# # save the model to disk
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

# y_pred = rf_model.predict(X_test_scaled)
# print(f'*****************RANDOM FOREST MODEL*******************')
# print(classification_report_imbalanced(y_test, y_pred))
# # rf_model.score(X_train_scaled, y_train)
# # rf_model.score(X_test_scaled, y_test)
# print(f'Training Score: {rf_model.score(X_train_scaled, y_train)}')
# print(f'Testing Score: {rf_model.score(X_test_scaled, y_test)}')

In [15]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
gbm = GradientBoostingClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
gbm_search = RandomizedSearchCV(estimator = gbm, param_distributions = random_grid, n_iter = 40, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
gbm_search.fit(X_train_sm, y_train_sm)
gbm_search.best_params_

Fitting 3 folds for each of 40 candidates, totalling 120 fits


{'n_estimators': 792,
 'min_samples_split': 3,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 7}

In [13]:
from imblearn.metrics import classification_report_imbalanced
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier
import pickle
#fit the model to the training set
gbm_model = GradientBoostingClassifier(random_state=42,n_estimators=215,
              min_samples_split=2,
              min_samples_leaf=5,
              max_features='sqrt',
              max_depth=4
              )

gbm_model.fit(X_train_sm, y_train_sm)
# save the model to disk
filename = 'finalized_model2.pkl'
pickle.dump(gbm_model, open(filename, 'wb'))

#Print Classification Report and Accuracy
y_pred = gbm_model.predict(X_test)
print(f'*****************************GRADIENT BOOSTING MODEL USING SMOTE************************************')
print(classification_report_imbalanced(y_test, y_pred))
# rf_model.score(X_train_scaled, y_train)
# rf_model.score(X_test_scaled, y_test)
print(f'Training Score: {gbm_model.score(X_train_sm, y_train_sm)}')
print(f'Testing Score: {gbm_model.score(X_test, y_test)}')

*****************************GRADIENT BOOSTING MODEL USING SMOTE************************************
                             pre       rec       spe        f1       geo       iba       sup

            Graduated       0.83      0.80      0.73      0.81      0.76      0.59       633
Terminated/Discharged       0.69      0.73      0.80      0.71      0.76      0.58       391

          avg / total       0.78      0.77      0.76      0.77      0.76      0.58      1024

Training Score: 0.8597005208333334
Testing Score: 0.7734375
